![title](quiz.png)

In [1]:
import numpy as np
import pulp as pl

pl.list_solvers(onlyAvailable=True)

['GLPK_CMD', 'GUROBI_CMD', 'PULP_CBC_CMD', 'PULP_CHOCO_CMD']

In [2]:
model = pl.LpProblem(name="quiz-1", sense=pl.LpMaximize)

x_vars = {}
for i in range(1, 3):
    x_vars[i] = pl.LpVariable(name=f"x_{i}", lowBound=0, cat="Continuous")

obj_func = x_vars[1] + 3 * x_vars[2]
model += obj_func

# Constraints
model += (-1 * x_vars[1] + x_vars[2] <= 3, "constraint_1")
model += (-1 * x_vars[1] + 2 * x_vars[2] <= 8, "constraint_2")
model += (3 * x_vars[1] + x_vars[2] <= 18, "constraint_3")

status = model.solve(solver=pl.PULP_CBC_CMD(msg=0))
if status == 1:
    print(model.objective.value())
    for var in model.variables():
        print(var.name, "=", var.value())

22.0
x_1 = 4.0
x_2 = 6.0


In [3]:
# Solve manually
A = np.array([
    [-1, 1, 1, 0, 0],
    [-1, 2, 0, 1, 0],
    [3, 1, 0, 0, 1],
])

b = np.array([3, 8, 18]).reshape((-1, 1))
c = np.array([1, 3, 0, 0, 0]).reshape((-1, 1))
m, n = A.shape

c_B_idx = [1, 3, 4]  # inital guess of the basis
c_N_idx = [x for x in range(n) if x not in c_B_idx]

c_B = np.array(c[c_B_idx])
c_N = np.array(c[c_N_idx])
A_B = A[:, c_B_idx]
A_N = A[:, c_N_idx]
reduced_costs = (c_B.T @ np.linalg.inv(A_B) @ A_N - c_N.T).T

print(A_N)
print(reduced_costs)

while reduced_costs.min() < 0:
    x_B = np.linalg.inv(A_B) @ b
    print(f"The basis for the bfs is {x_B.flatten()}, with indices of {c_B_idx}")
    # pick the entering variable
    enter_idx = c_N_idx[np.argmax(reduced_costs < 0)]
    print(f"The index of the entering variable is {enter_idx}")
    
    # ratio test
    left = np.linalg.inv(A_B) @ A[:, [enter_idx]]
    right = np.linalg.inv(A_B) @ b
    ratios = np.divide(right, left)
    
    # pick the leaving variable
    leaving_idx = c_B_idx[np.where(ratios > 0, ratios, np.inf).argmin()]
    print(f"The index of the leaving variable is {leaving_idx}\n")
    
    # construct new basis
    c_B_idx = [x for x in c_B_idx if x != leaving_idx] + [enter_idx]
    c_N_idx = [x for x in c_N_idx if x != enter_idx] + [leaving_idx]
    
    # update the values
    c_B = np.array(c[c_B_idx])
    c_N = np.array(c[c_N_idx])
    A_B = A[:, c_B_idx]
    A_N = A[:, c_N_idx]
    reduced_costs = (c_B.T @ np.linalg.inv(A_B) @ A_N - c_N.T).T
    
z = c_B.T @ np.linalg.inv(A_B) @ b
print(z)

[[-1  1]
 [-1  0]
 [ 3  0]]
[[-4.]
 [ 3.]]
The basis for the bfs is [ 3.  2. 15.], with indices of [1, 3, 4]
The index of the entering variable is 0
The index of the leaving variable is 3

The basis for the bfs is [5. 7. 2.], with indices of [1, 4, 0]
The index of the entering variable is 2
The index of the leaving variable is 4

[[22.]]
